# Learning and experiment notes on Python Standard Library.
## [Section 8.9](https://docs.python.org/3.6/library/types.html)

This module defines utility function to assist in dynamic creation of new types.

It also defines names for some object types that are used by the standard Python interpreter, but not exposed as builtins like int or str are.

Finally, it provides some additional type-related utility classes and functions that are not fundamental enough to be builtins.

## 8.9.1. Dynamic Type Creation

### (1) Intro
There are two functions, types.**new_class** and types.**prepare_class**:

`types.new_class(name, bases=(), kwds=None, exec_body=None)`

`types.prepare_class(name, bases=(), kwds=None)`

Lets's see how to use them

In [1]:
import types

In [2]:
a = types.new_class("programmer")

In [3]:
type(a)

type

In [4]:
jack = a()

In [5]:
type(jack)

types.programmer

We can see that `a` is a type, and `jack` is a `types.programmer` object.

In [6]:
type([])

list

In [7]:
type(list)

type

Similarly, `[]` is of built-in type `list`, whilst `list` is an instance of `type`.

__In fact, classes are instance of type, and metaclass is a subclass of type.__

### (2) Create class dynamically
Let's see what's the difference between `types.new_class` and the built-in `type`:

In [8]:
# Using type
programmer = type("Programmer",
                  (),
                  {"age": 20,
                   "name": "Jack",
                   "sayhi": (lambda self: "Hi, I'm {}, {} years old.".format(self.name, self.age))
                  })

In [9]:
programmer

__main__.Programmer

In [10]:
type(programmer)

type

In [11]:
jack = programmer()

In [12]:
type(jack)

__main__.Programmer

In [13]:
jack.sayhi()

"Hi, I'm Jack, 20 years old."

**Now, you may ask, how can we achieve the same functionality using `types.new_class` ?"**

Firstly, a metaclass should be defined:

In [14]:
class MetaCoder(type):
    def __new__(meta, classname, superclasses, attributedict):
        return type.__new__(meta, "Coder",
                            (),
                            {"age": 26,
                             "name": "William",
                             "sayhi": (lambda self: "Hi, I'm {}, {} years old.".format(self.name, self.age))
                            })

In the above code, our metaclass just reused `type.__new__` method with **hard coded** parameters. Now, let's use `types.new_class` to create a new class:

In [15]:
coder = types.new_class("Coder", (), {"metaclass": MetaCoder})

In [16]:
type(coder)

__main__.MetaCoder

In [17]:
will = coder()

In [18]:
type(will)

__main__.Coder

In [19]:
will.sayhi()

"Hi, I'm William, 26 years old."

Knowing the difference between the built-in `type` and `types.new_class`, one normally might ask 'Why bother using the `types.new_class` if we can directly use `type` and bypassing all the metaclass stuff? 

Before dispelling this doubt, one more little detail must be explicated.

### (3) How the two methods work?

For now, we only showed some examples of `types.new_class`, but what about `types.prepare_class`?

In [20]:
types.prepare_class("Coder", (), {"metaclass": MetaCoder})

(__main__.MetaCoder, {}, {})

The definition of `types.prepare_class` is as follows:
>Calculates the appropriate metaclass and creates the class namespace.

>The arguments are the components that make up a class definition header: the class name, the base classes (in order) and the keyword arguments (such as metaclass).

>The return value is a 3-tuple: metaclass, namespace, kwds

>metaclass is the appropriate metaclass, namespace is the prepared class namespace and kwds is an updated copy of the passed in kwds argument with any 'metaclass' entry removed. If no kwds argument is passed in, this will be an empty dict.

In Out[21] the metaclass is no suprising. Why are namespace and kwds empty?

In [21]:
types.prepare_class("Coder", (), {"metaclass": MetaCoder, "test_attr": True})

(__main__.MetaCoder, {}, {'test_attr': True})

Now we can pass additional class attribute, but namespace is still empty.

If you look at the [Lib/types.py](https://github.com/python/cpython/blob/3.3/Lib/types.py) source code, you will see that the `types.prepare_class` is called inside `types.new_class`. Unfortunately, here is how deep I can go into insofar. **I'm not sure what happend between line 80 and 84. **

Nevertheless, we can now draw a conclusion that `types.new_class` can be used to create new class dynamically with a metaclass defined.


## 8.9.2. Standard Interpreter Types

This sub-section listed some types in `types` module:
>This module provides names for many of the types that are required to implement a Python interpreter. It deliberately avoids including some of the types that arise only incidentally during processing such as the listiterator type.

Anyway, they seem not playable at the point I'm writing this note.

## -> More examples

For now, the examples given are far too simple. Some realistic scenarios should help.

### Case 1: 